<a href="https://colab.research.google.com/github/ednilsonmcs/analysis-pyspark/blob/main/TrabalhoPyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os

In [ ]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark

In [ ]:
import findspark 

In [ ]:
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql.types import IntegerType

In [ ]:
from pyspark.sql.functions import  month, year, to_date, date_format

In [ ]:
sc = SparkSession.builder.master('local[*]').getOrCreate() 

In [ ]:
sc.version

'3.1.2'

In [ ]:
dp = sc.read.options(header=True).csv(["/tmp/bombeiros"])

In [ ]:
dp = dp.withColumn("data_us",to_date(dp.Data, "dd/MM/yyyy"))

In [ ]:
dp = dp.withColumn("mes", month(dp.data_us))

In [ ]:
dp = dp.withColumn("ano", year(dp.data_us))

In [ ]:
dp = dp.withColumn("Valor Bruto (R$)", regexp_replace(dp['Valor Bruto (R$)'], "[^0-9.]", ""))

In [ ]:
dp = dp.withColumn("Valor Bruto (R$)", regexp_replace(dp['Valor Bruto (R$)'], "\\.", ""))

In [ ]:
dp = dp.withColumn("Valor Bruto (R$)", dp['Valor Bruto (R$)'].cast("float"))

In [ ]:
dp.groupBy("mes", "ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=False)

DataFrame[mes: int, ano: int, valor_total: double]

In [ ]:
rank_mes_maior_gastos = dp.groupBy("mes", "ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=False)

In [ ]:
rank_mes_menor_gastos = dp.groupBy("mes", "ano").agg(sum("Valor Bruto (R$)").alias("valor_total")).orderBy("valor_total", ascending=True)

In [ ]:
media_mensal_anual = dp.groupBy("ano").agg(avg("Valor Bruto (R$)").alias("media_mensal")).orderBy("ano", ascending=False)

In [ ]:
rank_mes_maior_gastos.show()

+---+----+-------------+
|mes| ano|  valor_total|
+---+----+-------------+
| 12|2017| 2.19374784E9|
| 10|2016| 1.26285459E9|
|  9|2021|1.089494821E9|
| 12|2019|1.064163513E9|
|  9|2022|1.050803322E9|
|  5|2019|1.046575362E9|
| 11|2021|1.017852704E9|
|  8|2022| 8.24018941E8|
| 11|2022| 8.21771702E8|
|  6|2018| 8.05480431E8|
| 12|2018| 7.68446952E8|
| 10|2021| 7.53034281E8|
|  8|2020| 7.34045971E8|
|  8|2021| 7.33022625E8|
| 10|2022|   7.299149E8|
| 12|2020| 7.01857916E8|
| 12|2021| 6.96631601E8|
|  7|2021|  6.9078686E8|
|  3|2021| 6.88715085E8|
|  4|2022| 6.86140606E8|
+---+----+-------------+
only showing top 20 rows



In [ ]:
rank_mes_menor_gastos.show()

+---+----+-----------+
|mes| ano|valor_total|
+---+----+-----------+
|  1|2011|        0.0|
|  1|2012|    63438.0|
|  1|2015|    70762.0|
|  3|2011|  5916602.0|
|  1|2014|  7200892.0|
|  1|2013|  7365170.0|
|  2|2011|  9015824.0|
|  4|2011|2.2368382E7|
|  9|2012|2.2551506E7|
| 10|2011|2.2824025E7|
|  9|2013|2.4033908E7|
|  6|2013|2.4296229E7|
|  2|2015|2.6648963E7|
|  6|2012|2.8415455E7|
|  9|2011|2.8885388E7|
|  2|2014|2.9264709E7|
|  8|2015|2.9680735E7|
|  2|2012| 3.018917E7|
| 10|2014|3.0708503E7|
|  7|2012|3.1313881E7|
+---+----+-----------+
only showing top 20 rows



In [ ]:
media_mensal_anual.show()

+----+--------------------+
| ano|        media_mensal|
+----+--------------------+
|2022|   8943865.834070796|
|2021|1.0236525438515082E7|
|2020|   7564575.443089431|
|2019|  6353167.4020715635|
|2018|   5347474.279965004|
|2017|   4662496.313159747|
|2016|  2473558.3476635516|
|2015|   627312.6907467532|
|2014|   655585.9293023256|
|2013|   488017.9815016322|
|2012|  459442.17279046675|
|2011|   526528.1503759399|
+----+--------------------+



In [ ]:
rank_mes_maior_gastos.repartition(1).write.csv("/tmp/output-rank_mes_maior_gastos")

In [ ]:
rank_mes_menor_gastos.repartition(1).write.csv("/tmp/output-rank_mes_menor_gastos")

In [ ]:
media_mensal_anual.repartition(1).write.csv("/tmp/output-media_mensal_anual")